In [2]:
import math

def wave(amplitude, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        randians = step * step_size
        fraction = math.sin(randians)
        output = amplitude * fraction
        yield output
        
def transmit(output):
    if output is None:
        print(f'Output is None')
    else:
        print(f'Output: {output:>5.1f}')

def run(it):
    for output in it:
        transmit(output)

run(wave(3.0, 8))        

Output:   0.0
Output:   2.1
Output:   3.0
Output:   2.1
Output:   0.0
Output:  -2.1
Output:  -3.0
Output:  -2.1


This works fine for producing basic waveforms, but it can't be used to constantly vary the amplitude of the wave based on a  separate input.

### Python generators support the send method, which upgrades yield expressions into a two-way channel. The send method can be used to provide streaming inputs to a generator at the same time it's yielding outputs

In [4]:
def my_generator():
    received = yield 1
    print(f'receivde = {received}')
it = iter(my_generator())
output = next(it)
print(f'output = {output}')
print('1')
try:
    next(it)
except StopIteration:
    pass
print('2')

output = 1
1
receivde = None
2


In [8]:
#when I call the send method instead of iterating the generator with a
#for loop or the next built-in function, the supplied parameter becomes
#the  value of the yield expression when the generator is resumed.

it = iter(my_generator())
#However, when the generator first starts, a yield expression has not been
#encountered yet, so the only valid value for calling send initially is None(any other
#argument would raise an exception at runtime)
output = it.send(None)     #Get first generator output
print(f'output = {output}')

try:
    it.send('hello!')      # Send value into the generator
except StopIteration:
    pass

TypeError: can't send non-None value to a just-started generator

In [9]:
def wave_modulating(steps):
    step_size = 2 * math.pi /steps
    amplitude = yield              #Receive initial amplitude
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        amplitude = yield output    # Receive next amplitude
        
def run_modulating(it):
    amplitudes = [
        None, 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    for amplitude in amplitudes:
        output = it.send(amplitude)
        transmit(output)
run_modulating(wave_modulating(12))

Output is None
Output:   0.0
Output:   3.5
Output:   6.1
Output:   2.0
Output:   1.7
Output:   1.0
Output:   0.0
Output:  -5.0
Output:  -8.7
Output: -10.0
Output:  -8.7
Output:  -5.0


In [11]:
def complex_wave():
    yield from wave(7.0, 3)
    yield from wave(2.0, 4)
    yield from wave(10.0, 5)
    
run(complex_wave())

Output:   0.0
Output:   6.1
Output:  -6.1
Output:   0.0
Output:   2.0
Output:   0.0
Output:  -2.0
Output:   0.0
Output:   9.5
Output:   5.9
Output:  -5.9
Output:  -9.5


In [12]:
def complex_wave_modulating():
    yield from wave_modulating(3)
    yield from wave_modulating(4)
    yield from wave_modulating(5)

run_modulating(complex_wave_modulating())

Output is None
Output:   0.0
Output:   6.1
Output:  -6.1
Output is None
Output:   0.0
Output:   2.0
Output:   0.0
Output: -10.0
Output is None
Output:   0.0
Output:   9.5
Output:   5.9


The easiest solution is to pass an iterator into the wave function. The iterator should return an input amplitude each time the next built-in function is called on it.

In [14]:
def wave_cascading(amplitude_it, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction =  math.sin(radians)
        amplitude = next(amplitude_it)  #get next input
        output = amplitude * fraction
        yield output
        
def complex_wave_cascading(amplitude_it):
    yield from wave_cascading(amplitude_it, 3)
    yield from wave_cascading(amplitude_it, 4)
    yield from wave_cascading(amplitude_it, 5)
    
def run_cascading():
    amplitudes = [ 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    it = complex_wave_cascading(iter(amplitudes))
    for _ in amplitudes:
        output = next(it)
        transmit(output)
        
run_cascading()

Output:   0.0
Output:   6.1
Output:  -6.1
Output:   0.0
Output:   2.0
Output:   0.0
Output:  -2.0
Output:   0.0
Output:   9.5
Output:   5.9
Output:  -5.9
Output:  -9.5
